In [3]:
# ✅ Dynamic Pricing for Urban Parking Lots — Google Colab Version

# --- Install and Import ---
!pip install pathway geopy bokeh

import numpy as np
import pandas as pd
import pathway as pw
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook
from google.colab import files
from geopy.distance import geodesic

output_notebook()

# --- Upload Data ---

df = pd.read_csv("dataset.csv")
df.head()

# --- Pricing Functions (Baseline, Demand, Competitive) ---

BASE_PRICE = 10

def baseline_linear_price(prev_price, occupancy, capacity, alpha=0.5):
    return prev_price + alpha * (occupancy / capacity)

def demand_based_price(base_price, occupancy, capacity, queue_length, traffic, is_special_day, vehicle_weight,
                       alpha=0.4, beta=0.3, gamma=0.2, delta=0.1, epsilon=0.1, lambda_=0.5):
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue_length -
        gamma * traffic +
        delta * is_special_day +
        epsilon * vehicle_weight
    )
    demand = np.clip(demand, 0, 1)
    price = base_price * (1 + lambda_ * demand)
    return np.clip(price, 0.5 * base_price, 2 * base_price)

def competitive_price(current_price, competitor_prices, lot_location, competitor_locations, is_full):
    nearby = []
    for p, loc in zip(competitor_prices, competitor_locations):
        if geodesic(lot_location, loc).meters < 500:
            nearby.append(p)
    if not nearby:
        return current_price
    avg_nearby = np.mean(nearby)
    if is_full and current_price > avg_nearby:
        return avg_nearby * 0.9
    elif current_price < avg_nearby:
        return current_price * 1.1
    return current_price

# --- Example Real-Time Simulation (Simple Loop) ---

prices = []
prev_price = BASE_PRICE
for i in range(20):
    occ = np.random.randint(0, 100)
    cap = 100
    price = baseline_linear_price(prev_price, occ, cap)
    prices.append(price)
    prev_price = price

p = figure(title="Price Simulation", x_axis_label="Time Step", y_axis_label="Price ($)")
p.line(list(range(len(prices))), prices, line_width=2)
show(p)

# --- Add your Pathway streaming hooks here to replace loop ---
